In [1]:
import requests
import pandas as pd
import threading
import json
import datetime
import time
import os
import numpy as np

def get_size(size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page=0&search=&size={}&sortOrder=desc&sortProperty=apr".format(size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['total']

def get_validator(page, size):
    html_url = "https://staking-explorer2-268108.appspot.com/networks/harmony/validators_with_page?active=false&page={}&search=&size={}&sortOrder=desc&sortProperty=apr".format(page, size)
    res = requests.get(html_url)
    content = json.loads(res.content)
    return content['validators']

def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        content = json.loads(r.content)
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    if "error" in content:
        print("Error: The method does not exist/is not available")
        return None
    else:
        return content['result']

def getAllValidatorInformation():
    url = 'https://api.s0.t.hmny.io/'
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(url, method, params)


In [2]:
val_infos = getAllValidatorInformation()
address = []
name = []
apr = []
uptime = []
curr_uptime = []
stake = []
self_stake = []
rate = []
epos_status = []
boot_status = []
for i in val_infos:
    address.append(i['validator']['address'])
    name.append(i['validator']['name'])
    apr.append("{0:.2f}%".format(float(i['lifetime']['apr'])*100))
    sign_info = i['lifetime']['blocks']
    if float(sign_info['to-sign']) != 0:
        sign_perc = float(sign_info['signed'])/float(sign_info['to-sign'])
        uptime.append("{0:.2f}%".format(sign_perc*100))
    else:
        uptime.append(None)   
    if i['current-epoch-performance']:
        curr_perc = float(i['current-epoch-performance']['current-epoch-signing-percent']['current-epoch-signing-percentage'])
        curr_uptime.append("{0:.2f}%".format(curr_perc*100))
    else:
        curr_uptime.append(None)
    stake.append(int(float(i['total-delegation'])/1e18))
    self_stake.append(i['validator']['delegations'][0]['amount']/1e18)
    rate.append("{0:.2f}%".format(float(i['validator']['rate'])))
    epos_status.append(i['epos-status'])
    boot_status.append(i['booted-status'])
df = pd.DataFrame(list(zip(address, name, apr, stake, self_stake, rate, uptime, curr_uptime, epos_status, boot_status)), columns =['address', 'name', 'apr', 'total-stake', 'self-stake', 'fees', 'history_uptime', 'curr_epoch_uptime','epos-status','boot_status'])


In [3]:
df

,address,name,apr,total-stake,self-stake,fees,history_uptime,curr_epoch_uptime,epos-status,boot_status
0,one1hwe68yprkhp5sqq5u7sm9uqu8jxz87fd7ffex7,CONTABO,57.75%,152619,1.817300e+04,0.00%,94.26%,None,eligible to be elected next epoch,lost epos auction
1,one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj,Sesameseed,10.84%,123984135,3.000001e+06,1.00%,98.55%,99.82%,currently elected,None
2,one1h9z66rjlrely889rkhlaxf7chk3pv2ksy46eq7,CONTABO-2,102.47%,156140,8.000000e+04,0.00%,99.76%,None,eligible to be elected next epoch,lost epos auction
3,one1kyyt7j29h4uhtnuhfar5wmngntx4gterrkd8q9,Total Harmony,10.91%,17279759,1.249900e+06,0.04%,98.99%,100.00%,currently elected,None
4,one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj,🔨 P-OPS Team Validator | Community Run,10.86%,320427991,1.228270e+05,0.05%,99.92%,100.00%,currently elected,None
...,...,...,...,...,...,...,...,...,...,...
248,one13qq8ewl349drehwg73038gfknxjuc2nvm2uwny,Ankr-czone-1,0.00%,20500,1.550000e+04,0.10%,None,None,eligible to be elected next epoch,not booted
249,one1n9vm6kg26zf2tp65t227s647lrv75343aun3nr,DeeCrypt_ONE,10.81%,7756571,1.060700e+04,0.05%,99.68%,98.90%,currently elected,None
250,one14hhq3p6t86za3aefvwfklwtlufjf06ymf3e2mh,Harmony Korea,0.00%,16640,1.664032e+04,0.01%,None,None,eligible to be elected next epoch,not booted
251,one1zgmm8dwyzffsa7379ghdkgt5e6tsm4tpl50arr,The Harmony Bakery,0.00%,321700,1.000000e+04,0.07%,None,None,eligible to be elected next epoch,not booted
